# 02: Line Segment Intersection

*Authors: Jan Erik Swiadek, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases implementations of algorithms and data structures presented in the lecture, and it elaborates on some practical considerations concerning their use.
Furthermore, it provides interactive visualisations and animations.

## Table of Contents

1. Setup  
2. Introduction  
3. Algorithms  
    3.1. Brute Force  
    3.2. Plane Sweep  
4. References  

## 1. Setup

First let's import everything we'll need throughout this notebook, including our module for generic data structures, our module for common geometry primitives and operations as well as our module for visualisation purposes. (The data structure and geometry modules will probably receive their own notebook later.)

This is not very interesting, so you can skip straight to section 2.

In [ ]:
# Python standard library imports
from typing import Any, Optional
from itertools import combinations
import os
import sys

# Make our modules available for importing
modules_dir = os.path.abspath("../modules")
if modules_dir not in sys.path:
    sys.path.append(modules_dir)

# xxx module imports
from data_structures import BinaryTree, BinaryTreeDict, Comparator, ComparisonResult as CR
from geometry import Orientation as ORT, Point, LineSegment, Intersections
from visualisation import Visualisation, LineSegmentSetInstance, PointsMode, SweepLineMode

Additionally, we create a Visualisation object and register a few example instances for it.

TODO: Add useful example instances.

In [ ]:
visualisation = Visualisation(400, 400, LineSegmentSetInstance())

seg1 = LineSegment(Point(150, 25), Point(100, 300))
seg2 = LineSegment(Point(75, 150), Point(200, 250))
seg3 = LineSegment(Point(50, 250), Point(300, 125))
seg4 = LineSegment(Point(200, 100), Point(350, 225))
visualisation.register_example_instance("test", set([seg1, seg2, seg3, seg4]))

hor = set([
    LineSegment(Point(150, 200), Point(225, 200)),
    LineSegment(Point(175, 200), Point(250, 200)),
    LineSegment(Point(175, 175), Point(225, 225)),
    LineSegment(Point(125, 125), Point(215, 215)),
    LineSegment(Point(125, 150), Point(175, 150))
])
visualisation.register_example_instance("horizontal/overlap", hor)

mult = set([
    LineSegment(Point(150, 200), Point(250, 200)),
    LineSegment(Point(175, 175), Point(225, 225)),
    LineSegment(Point(200, 150), Point(200, 250)),
    LineSegment(Point(175, 225), Point(225, 175))
])
visualisation.register_example_instance("multiple", mult)

## 2. Introduction

The **line segment intersection problem** was stated in the lecture as follows:
Given a set of $n$ line segments in the plane, the goal is to compute all points in the intersection of two or more line segments along with these respective segments.
Only **closed line segments** are considered, i.e. each segment contains two bounding endpoints.
The endpoints are therefore potential intersection points just like any other point contained in the segments.

For example, the intersections of the xxx instance are marked here:

TODO: Include image.

## 3. Algorithms

The lecture presented two algorithms for the problem:
A simple Brute Force algorithm and, as the main focus of the lecture, a more sophisticated Plane Sweep method.

### 3.1 Brute Force

The easiest way to solve the line segment intersection problem is to test each pair of line segments in the input for their intersection.
There are three possibilities for the intersection of two line segments:
They don't intersect, in which case nothing needs to be done, or they intersect in exactly one point, which is then added to the output together with the segments, or they're **overlapping line segments** and their intersection is again a line segment.
As we would technically need to report infinitely many intersection points in the third case, it's considered degenerate and we choose to report just the endpoints.

In [ ]:
def brute_force_lsi(segments: set[LineSegment]) -> Intersections:
    intersections = Intersections()
    for segment1, segment2 in combinations(segments, 2):
        intersection = segment1.intersection(segment2)
        if isinstance(intersection, Point):
            intersections.add(intersection, (segment1, segment2))
        elif isinstance(intersection, LineSegment):
            intersections.add(intersection.upper, (segment1, segment2))
            intersections.add(intersection.lower, (segment1, segment2))
    
    return intersections

The asymptotic running time of Brute Force is obviously in $\Theta(n^2)$.
This is worst-case optimal, with the worst case being that all line segment pairs intersect in one point.
However, the algorithm performs many "unnecessary" tests if the input line segments have few intersection points.

We now register Brute Force for visualisation.

In [ ]:
visualisation.register_algorithm("Brute Force", brute_force_lsi, PointsMode())

If you haven't used our interactive visualisation tool before, see the Convex Hull notebook for an explanation.

In [ ]:
visualisation.display()

TODO: Include image.

***Takeaways:***

* The simplest algorithm can already be worst-case optimal.

* In practice, the worst case isn't the only case to consider.

### 3.2 Plane Sweep

Considering that for most practical puporses only few intersections occur in relation to the number of input segments, an output-sensitive algorithm that depends on the number of intersections is of interest.
We've already seen examples for such algorithms: Gift Wrapping and Chan's Hull for the convex hull problem.
The output-sensitive algorithm from the line segment intersection lecture uses the **plane sweep technique**, which is a general technique suitable to solving various problems in computational geometry.
It simulates a **sweep line** to pass through the input elements, while maintaining the status at the current sweep line position and the event points that are yet to be visited by the sweep line.
The versatility of this technique comes from the fact that new event points can be computed on the fly during the course of the sweep.

Here, the **status structure** consists of all line segments currently intersected by the sweep line, while the **event queue** contains line segment endpoints and previously computed intersection points.
Like suggested in the lecture, we implement both as a *balanced binary tree*.
The searchable position of a tree node is determined by its *key*.
We define an order on the keys in the form of a *comparator* class, whose objects have a *compare(item, key)* method comparing an *item* (e.g. a new key to be inserted) to an existing node key.
The return value specifies whether the item comes before, matches with or comes after the key.

The keys of the event queue are event points, and items to be compared with them are also always points.
Since we want the sweep line to go from top to bottom, we prioritise points with a greater $y$-coordinate.
Such points should thus come first in the event queue order (see [1, p. 24]).
Moreover, we deal with the degenerate case of event points having the same $y$-coordinate by prioritising smaller $x$-coordinates, i.e. the corresponding events are handled from left to right.

In [ ]:
class EventQueueComparator(Comparator[Point]):
    def compare(self, item: Any, key: Point) -> CR:
        if not isinstance(item, Point):
            raise TypeError("Only points can be compared with event points.")
        if item == key:
            return CR.MATCH
        elif item.y > key.y or (item.y == key.y and item.x < key.x):
            return CR.BEFORE
        else:
            return CR.AFTER

The comparator class for the status structure is less straightforward.
Because the left-to-right order of stored line segments depends on the current height of the sweep line and can change at event points, a *dynamic comparator* is required (see [2, pp. 24–25]).
The keys of the status stucture are line segments, which are compared to other line segments at the current sweep line height for tree insertion and deletion.
Additionally, the node keys also need to be comparable with points in order to search for all segments containing the current event point and to determine its neighbouring segments.
Therefore the comparator has to support points as items too.

Comparing a point to a line segment is easy using an orientation test.
The comparison of two line segments can then be reduced to finding the intersection point of one line segment with the current sweep line and comparing this point to the other segment.
If this point is contained in the other segment as well, the location of the event point becomes relevant.
That's also true for horizontal line segments, which constitute a degenerate case and are dealt with separately.
In the other degenerate case we've mentioned before, two overlapping line segments, the order doesn't actually matter as long as it's consistent, so we simply use their coordinates tuples.

In [ ]:
class StatusStructureComparator(Comparator[LineSegment]):
    def __init__(self):
        self._event_point: Optional[Point] = None

    def set_event_point(self, event_point: Point):
        self._event_point = event_point

    def compare(self, item: Any, key: LineSegment) -> CR:
        if isinstance(item, Point):
            return self._compare_point_with_segment(item, key)
        elif isinstance(item, LineSegment):
            return self._compare_segment_with_segment(item, key)
        else:
            raise TypeError("Only line segments and points can be compared with status line segments.")

    def _compare_point_with_segment(self, point: Point, segment: LineSegment) -> CR:
        if segment.upper.y < point.y or segment.lower.y > point.y:
            raise ValueError(f"Point {point} isn't in y-range of compared segment {segment}.")
        elif segment.upper.y == segment.lower.y:
            if point.x < segment.upper.x:
                return CR.BEFORE
            elif point.x > segment.lower.x:
                return CR.AFTER
            else:
                return CR.MATCH
        else:
            ort = point.orientation(segment.lower, segment.upper)
            if ort is ORT.LEFT:
                return CR.BEFORE
            elif ort is ORT.RIGHT:
                return CR.AFTER
            else:
                return CR.MATCH

    def _compare_segment_with_segment(self, segment1: LineSegment, segment2: LineSegment) -> CR:
        special_cr = self._check_special_cases(segment1, segment2)
        if special_cr is not None:
            return special_cr
        else:
            left_point = Point(min(segment1.upper.x, segment1.lower.x) - 1.0, self._event_point.y)
            right_point = Point(max(segment1.upper.x, segment1.lower.x) + 1.0, self._event_point.y)
            segment1_point = segment1.intersection(LineSegment(left_point, right_point))
            segment1_point_cr = self._compare_point_with_segment(segment1_point, segment2)
            if segment1_point_cr is not CR.MATCH:
                return segment1_point_cr
            else:
                event_point_cr = self._compare_point_with_segment(self._event_point, segment2)
                before_ort = ORT.RIGHT if event_point_cr is CR.BEFORE else ORT.LEFT
                if segment1.lower.orientation(segment2.lower, self._event_point) is before_ort:
                    return CR.BEFORE
                else:
                    return CR.AFTER

    def _check_special_cases(self, segment1: LineSegment, segment2: LineSegment) -> Optional[CR]:
        if self._event_point is None:
            raise RuntimeError("Event point has to be set for line segment comparison")
        elif segment1.upper.y < self._event_point.y or segment1.lower.y > self._event_point.y:
            raise ValueError(f"Event point {self._event_point} isn't in y-range of compared segment {segment1}.")
        elif segment2.upper.y < self._event_point.y or segment2.lower.y > self._event_point.y:
            raise ValueError(f"Event point {self._event_point} isn't in y-range of compared segment {segment2}.")
        elif segment1 == segment2:
            return CR.MATCH
        elif isinstance(segment1.intersection(segment2), LineSegment):
            segment1_tuple = (segment1.upper.x, segment1.upper.y, segment1.lower.x, segment1.lower.y)
            segment2_tuple = (segment2.upper.x, segment2.upper.y, segment2.lower.x, segment2.lower.y)
            if segment1_tuple < segment2_tuple:
                return CR.BEFORE
            else:
                return CR.AFTER
        elif segment1.upper.y == segment1.lower.y:
            if self._compare_point_with_segment(self._event_point, segment2) is CR.BEFORE:
                return CR.BEFORE
            else:
                return CR.AFTER
        elif segment2.upper.y == segment2.lower.y:
            if self._compare_point_with_segment(self._event_point, segment1) is CR.BEFORE:
                return CR.AFTER
            else:
                return CR.BEFORE
        else:
            return None

Now we can implement the plane sweep algorithm itself.
We follow the approach from [1, pp. 26–27].

At first, the status structure is initialised as an empty binary tree, while the event queue is initialised as a binary tree containing all line segment endpoints as keys.
The event queue also stores a *value* for each key that consists of the list of line segments having this key as their upper endpoint.
That's why the event queue's type is *BinaryTreeDict* as opposed to the simple *BinaryTree* type used for the status structure, which doesn't support values.

During each step of the actual sweep, an event point is popped from the queue along with the segments having it as their upper endpoint.
A subsequent search in the status structure yields all other line segments containing the event point, which are promptly deleted from the status structure.
Next, those segments having the event point as their upper endpoint or containing it in their interior are inserted into the status structure.
This is done according to the new status structure order induced by the event point, as implemented in the comparator above.
Note that segments containing the event point in their interior are deleted and re-inserted, so their order in the status structure is reversed.
(Except for the relative order of overlapping line segments, which doesn't matter as stated above.)
Hence, there's no need for a separate swap procedure and the degenerate case of more than 2 line segments intersecting in one point is handled elegantly (see [1, p. 26]).

Then the segments that have become adjacent in the status structure due to the deletions and (re-)insertions are checked for intersections.
If there now aren't any line segments in the status structure that contain the event point, the left and right status neighbours of the event point are adjacent.
Otherwise, the left status neighbour is adjacent to the leftmost containing segment in the status structure, whereas the right status neighbour is adjacent to the rightmost such segment.
In both cases an intersection point of two newly adjacent segments is added to the event queue if it hasn't already been visited by the sweep line.

Finally, the event point is reported along with all its containing line segments.
If it has at least two such segments, then it's an intersection point.

In [ ]:
def plane_sweep_lsi(segments: set[LineSegment]) -> Intersections:
    return PlaneSweepLSI(segments).sweep()

class PlaneSweepLSI:
    def __init__(self, segments: set[LineSegment]):
        self._status_structure_comparator = StatusStructureComparator()
        self._status_structure: BinaryTree[LineSegment] = BinaryTree(self._status_structure_comparator)

        self._event_queue_comparator = EventQueueComparator()
        self._event_queue: BinaryTreeDict[Point, list[LineSegment]] = BinaryTreeDict(self._event_queue_comparator)

        self._default_value_updater = lambda u_segments: [] if u_segments is None else u_segments
        for segment in segments:
            def upper_endpoint_value_updater(u_segments: Optional[list[LineSegment]]) -> list[LineSegment]:
                u_segments = self._default_value_updater(u_segments)
                u_segments.append(segment)
                return u_segments
            self._event_queue.update(segment.upper, upper_endpoint_value_updater)
            self._event_queue.update(segment.lower, self._default_value_updater)

    def sweep(self) -> Intersections:
        intersections = Intersections()
        while not self._event_queue.is_empty():
            event_point, containing_segments = self._handle_event()
            if len(containing_segments) >= 2:
                intersections.add(event_point, containing_segments)
            else:
                intersections.animate(event_point)      # Also animate event points that aren't intersection points.

        return intersections

    def _handle_event(self) -> tuple[Point, list[LineSegment]]:
        event_point, containing_segments = self._event_queue.pop_first()
        for segment in self._status_structure.search_matching(event_point):
            self._status_structure.delete(segment)
            containing_segments.append(segment)

        self._status_structure_comparator.set_event_point(event_point)

        for segment in filter(lambda segment: segment.lower != event_point, containing_segments):
            self._status_structure.insert(segment)

        left_status_neighbour = self._status_structure.search_predecessor(event_point)
        containing_status_segments = self._status_structure.search_matching(event_point)
        right_status_neighbour = self._status_structure.search_successor(event_point)

        if not containing_status_segments:
            if left_status_neighbour is not None and right_status_neighbour is not None:
                self._find_new_event(left_status_neighbour, right_status_neighbour, event_point)
        else:
            #left_status_neighbour = self._status_structure.search_predecessor(containing_status_segments[0])
            #right_status_neighbour = self._status_structure.search_successor(containing_status_segments[-1])
            if left_status_neighbour is not None:
                self._find_new_event(left_status_neighbour, containing_status_segments[0], event_point)
            if right_status_neighbour is not None:
                self._find_new_event(containing_status_segments[-1], right_status_neighbour, event_point)

        return event_point, containing_segments

    def _find_new_event(self, left_segment: LineSegment, right_segment: LineSegment, event_point: Point):
        intersection = left_segment.intersection(right_segment)
        if isinstance(intersection, Point):
            if self._event_queue_comparator.compare(intersection, event_point) is CR.AFTER:
                self._event_queue.update(intersection, self._default_value_updater)


The running time of Plane Sweep is in $O((n + i) \log (n))$ and the required space of the data structures is in $O(n + i)$, where $i$ is the number of intersections.
Note that the full size of the output is greater than $i$ because the intersecting segments are part of the output as well (see [1, p. 28]).

In [ ]:
visualisation.register_algorithm("Plane Sweep", plane_sweep_lsi, SweepLineMode())

The animation of this algorithm shows how the sweep line passes over the event points from top to bottom.

In [ ]:
visualisation.display()

TODO: Actually add sweep line to animation and include image.

TODO: Add takeaways and maybe talk about robustness.

***Takeaways:***

* ...

# TESTS

In [ ]:
seg1 = LineSegment(Point(6, 1), Point(4, 12))
seg2 = LineSegment(Point(3, 6), Point(8, 10))
seg3 = LineSegment(Point(2, 10), Point(12, 5))
seg4 = LineSegment(Point(8, 4), Point(14, 9))

segments = set([seg1, seg2, seg3, seg4])

print(seg1.intersection(seg2))
print()

print(brute_force_lsi(segments))
print()

intersections = PlaneSweepLSI(segments).sweep()
print(intersections)
#print(intersections._animation_events)

In [ ]:
print(brute_force_lsi(hor))
print()
print(PlaneSweepLSI(hor).sweep())

In [ ]:
print(PlaneSweepLSI(mult).sweep())

In [ ]:
print(LineSegment(Point(1,3), Point(3,1)).intersection(LineSegment(Point(1,1), Point(3,3))))
print(LineSegment(Point(1,3), Point(2,2)).intersection(LineSegment(Point(1,1), Point(3,3))))

print(LineSegment(Point(1,3), Point(1.5,2.5)).intersection(LineSegment(Point(1,1), Point(3,3))))
print(LineSegment(Point(4,6), Point(6,4)).intersection(LineSegment(Point(1,1), Point(3,3))))

print(LineSegment(Point(1,3), Point(3,1)).intersection(LineSegment(Point(0,2), Point(2,0))))
print(LineSegment(Point(1,3), Point(3,1)).intersection(LineSegment(Point(4,0), Point(5,-1))))

print(LineSegment(Point(1,3), Point(3,1)).intersection(LineSegment(Point(3,1), Point(4,0))))
print(LineSegment(Point(1,3), Point(3,1)).intersection(LineSegment(Point(2,2), Point(4,0))))
print(LineSegment(Point(1,3), Point(3,1)).intersection(LineSegment(Point(2,2), Point(0,4))))
print(LineSegment(Point(1,3), Point(3,1)).intersection(LineSegment(Point(2,2), Point(1.5,2.5))))

In [ ]:
point = seg1.intersection(seg2)

print(point)

comp = StatusStructureComparator()
print(comp.compare(point, seg1))
print(comp.compare(point, seg2))
print()

import numpy as np

for _ in range(100000):
    points = [Point(np.random.uniform(0, 400), np.random.uniform(0, 400)) for _ in range(4)]
    if points[0] == points[1] or points[2] == points[3]:
        continue
    segment1 = LineSegment(points[0], points[1])
    segment2 = LineSegment(points[2], points[3])
    intersection = segment1.intersection(segment2)
    if isinstance(intersection, Point):
        try:
            comp1 = comp.compare(intersection, segment1)
        except:
            print(f"Not comparable: {segment1} and {segment2}")
        if comp1 is not CR.MATCH:
            print(f"Not equal: {segment1} and {segment2}")
        try:
            comp2 = comp.compare(intersection, segment2)
        except:
            print(f"Not comparable: {segment1} and {segment2}")
        if comp2 is not CR.MATCH:
            print(f"Not equal: {segment1} and {segment2}")

## 4. References

[1] Mark de Berg, Otfried Cheong, Marc van Kreveld, and Mark Overmars. *Computational Geometry: Algorithms and Applications*, 3rd Edition, 2008.

[2] David M. Mount. [*CMSC 754: Computational Geometry*](https://www.cs.umd.edu/class/spring2020/cmsc754/Lects/cmsc754-spring2020-lects.pdf), 2020.